In [ ]:
!pip install nibabel ipywidgets matplotlib scikit-learn tensorflow opencv-python plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00


In [8]:
import os
import time
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from joblib import load
import tensorflow as tf
from matplotlib.patches import Circle
import cv2
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

# === Load Models ===
svm_bundle = load("svm_model_with_scaler.joblib")
svm_model = svm_bundle['model']
scaler = svm_bundle['scaler']
unet_model = tf.keras.models.load_model("GP-4_model.h5", compile=False)

# === Preprocessing ===
def preprocess_mri(file_path):
    img = nib.load(file_path).get_fdata()
    z_center = img.shape[2] // 2
    cropped = img[:, :, z_center - 32:z_center + 32] if img.shape[2] >= 64 else img
    resized = np.zeros((128, 128, 64))
    for i in range(min(64, cropped.shape[2])):
        resized[:, :, i] = cv2.resize(cropped[:, :, i], (128, 128), interpolation=cv2.INTER_NEAREST)
    norm = (resized - resized.min()) / (resized.max() - resized.min() + 1e-8)
    return norm[..., np.newaxis], img

# === Analyze Segmentation ===
def analyze_segmentation(segmentation, threshold=0.5):
    mask = segmentation > threshold
    volume_voxels = np.sum(mask)
    volume_cm3 = volume_voxels / 1000
    coords = np.array(np.where(mask))
    if coords.shape[1] == 0:
        return 0, 0, (0, 0, 0), 0
    centroid = tuple(np.mean(coords, axis=1))
    diameter = np.max(np.ptp(coords, axis=1))
    return volume_voxels, volume_cm3, centroid, diameter

# === GUI Class ===
class TumorAnalyzerGUI:
    def __init__(self):
        self.model = unet_model
        self.svm_model = svm_model
        self.scaler = scaler
        self.original = None
        self.preprocessed = None
        self.segmentation = None
        self.centroid = (0, 0, 0)
        self.create_widgets()
        self.build_gui()

    def create_widgets(self):
        self.upload = widgets.FileUpload(accept='.nii,.nii.gz', multiple=False, description="Upload MRI")
        self.run_btn = widgets.Button(description="⚡ Run Fast", button_style='success')
        self.reset_btn = widgets.Button(description="Reset", button_style='warning')
        self.view_3d_btn = widgets.Button(description="3D Tumor View", button_style='info')
        self.status = widgets.HTML(value="Status: Ready")
        self.results = widgets.HTML(value="Results will appear here.")
        self.output = widgets.Output()
        self.slice_slider = widgets.IntSlider(description='Slice', min=0, max=63)
        self.progress = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', bar_style='info')

        self.run_btn.on_click(self.run_analysis)
        self.reset_btn.on_click(self.reset_gui)
        self.view_3d_btn.on_click(self.show_3d_tumor)
        self.slice_slider.observe(self.update_view, names='value')

    def build_gui(self):
        display(widgets.VBox([
            widgets.HBox([self.upload, self.run_btn, self.reset_btn, self.view_3d_btn]),
            self.status,
            self.progress,
            self.results,
            self.slice_slider,
            self.output
        ]))

    def reset_gui(self, b=None):
        self.original = None
        self.preprocessed = None
        self.segmentation = None
        self.centroid = (0, 0, 0)
        self.status.value = "Status: Ready"
        self.results.value = "Results will appear here."
        self.output.clear_output()
        self.progress.value = 0
        self.progress.bar_style = 'info'

    def run_analysis(self, b=None):
        with self.output:
            clear_output()
            start = time.time()
            self.progress.value = 10
            if not self.upload.value:
                self.status.value = "⚠️ Please upload an MRI file."
                return

            file_data = next(iter(self.upload.value.values()))
            file_path = os.path.join("/tmp", file_data['metadata']['name'])
            with open(file_path, 'wb') as f:
                f.write(file_data['content'])

            self.status.value = "⏳ Preprocessing MRI..."
            self.progress.value = 20
            self.preprocessed, self.original = preprocess_mri(file_path)
            if self.preprocessed is None:
                self.status.value = "❌ Failed to preprocess image."
                return

            self.status.value = "⏳ Predicting segmentation..."
            self.progress.value = 40
            pred = self.model.predict(np.expand_dims(self.preprocessed, axis=0))[0, ..., 0]
            self.segmentation = pred
            voxels, vol_cm3, self.centroid, diameter = analyze_segmentation(pred)

            self.status.value = "⏳ Classifying tumor type..."
            self.progress.value = 70
            mask = pred > 0.5
            intensity_values = self.preprocessed[mask]

            if intensity_values.size == 0:
                self.status.value = "⚠️ No tumor detected."
                self.progress.bar_style = 'danger'
                return

            # CLASSIFICATION
            features = intensity_values.reshape(-1, 1)
            features_scaled = self.scaler.transform(features)
            preds = self.svm_model.predict(features_scaled)
            probs = self.svm_model.predict_proba(features_scaled)

            tumor_type = int(np.round(np.mean(preds)))
            confidence = np.max(np.mean(probs, axis=0))
            elapsed = time.time() - start

            # LOCATION INTERPRETATION
            z = self.centroid[2]
            x = self.centroid[0]
            y = self.centroid[1]

            z_desc = "upper" if z < 21 else "middle" if z < 43 else "lower"
            x_desc = "left lobe" if x < 64 else "right lobe"
            location_text = f"Tumor is likely located in the {x_desc}, {z_desc} region of the liver."

            self.progress.value = 100
            self.progress.bar_style = 'success'
            self.results.value = f"""
            <b>Tumor Classification:</b> {'Malignant' if tumor_type == 1 else 'Benign'}<br>
            <b>Confidence:</b> {confidence:.2%}<br>
            <b>Volume:</b> {vol_cm3:.2f} cm³ ({voxels} voxels)<br>
            <b>Centroid:</b> {tuple(map(lambda x: round(x, 1), self.centroid))}<br>
            <b>Max Diameter:</b> {diameter:.1f} voxels<br>
            <b>Location:</b> {location_text}<br>
            ⏱️ <b>Run Time:</b> {elapsed:.2f} sec
            """
            self.status.value = "✅ Analysis Complete."
            self.slice_slider.value = int(self.centroid[2])
            self.update_view()

    def update_view(self, change=None):
        with self.output:
            clear_output(wait=True)
            if self.original is None or self.segmentation is None:
                return
            idx = self.slice_slider.value
            fig, ax = plt.subplots(1, 2, figsize=(12, 5))
            ax[0].imshow(self.original[:, :, idx], cmap='gray')
            ax[0].set_title("Original")
            ax[0].axis("off")
            ax[1].imshow(self.original[:, :, idx], cmap='gray')
            ax[1].imshow(self.segmentation[:, :, idx], alpha=0.4, cmap='Reds')
            if int(round(self.centroid[2])) == idx:
                ax[1].add_patch(Circle((self.centroid[1], self.centroid[0]), 8, color='red', fill=False, lw=2))
            ax[1].set_title("Segmentation")
            ax[1].axis("off")
            plt.tight_layout()
            plt.show()

    def show_3d_tumor(self, b=None):
        with self.output:
            clear_output(wait=True)

            if self.segmentation is None:
                self.status.value = "⚠️ Run the analysis first to generate segmentation."
                return

            mask = self.segmentation > 0.5
            coords = np.argwhere(mask)

            if coords.size == 0:
                self.status.value = "⚠️ No tumor detected. Cannot display 3D view."
                return

            fig = go.Figure(data=[go.Scatter3d(
                x=coords[:, 0],
                y=coords[:, 1],
                z=coords[:, 2],
                mode='markers',
                marker=dict(size=2, color=coords[:, 2], colorscale='Reds', opacity=0.5)
            )])
            fig.update_layout(
                title="3D Tumor Visualization",
                scene=dict(
                    xaxis_title='X',
                    yaxis_title='Y',
                    zaxis_title='Z'
                )
            )
            fig.show()
            self.status.value = "🧠 3D tumor visualization rendered successfully."

# 🚀 Launch the GUI
TumorAnalyzerGUI()
